# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 2:40PM,253941,22,122822,"NBTY Global, Inc.",Released
1,Dec 28 2022 2:36PM,253938,19,8960258,"Avet Pharmaceuticals Labs, Inc",Released
2,Dec 28 2022 2:27PM,253937,19,8960256,"Avet Pharmaceuticals Labs, Inc",Released
3,Dec 28 2022 2:23PM,253936,10,Clear-8960252,"ClearSpec, LLC",Released
4,Dec 28 2022 2:17PM,253935,19,ACG-2102492259,ACG North America LLC,Released
5,Dec 28 2022 1:57PM,253934,19,N17889,"NAPP Technologies, LLC",Released
6,Dec 28 2022 1:56PM,253933,19,8960248,Paragon BioTeck,Released
7,Dec 28 2022 1:32PM,253930,10,MSP217068,"Methapharm, Inc.",Released
8,Dec 28 2022 1:28PM,253929,10,Enova-11148,Emerginnova,Released
9,Dec 28 2022 1:24PM,253927,10,Yusen-8960238,Yusen – 3D Matrix,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,253935,Released,1
42,253936,Released,1
43,253937,Released,1
44,253938,Released,1
45,253941,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253935,NaN,NaN,1.0
253936,NaN,NaN,1.0
253937,NaN,NaN,1.0
253938,NaN,NaN,1.0
253941,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253853,0.0,0.0,1.0
253855,0.0,0.0,5.0
253856,8.0,0.0,4.0
253862,0.0,0.0,1.0
253863,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253853,0,0,1
253855,0,0,5
253856,8,0,4
253862,0,0,1
253863,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253853,0,0,1
1,253855,0,0,5
2,253856,8,0,4
3,253862,0,0,1
4,253863,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253853,,,1
1,253855,,,5
2,253856,8,,4
3,253862,,,1
4,253863,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc."
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc"
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc"
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC"
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC
5,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC"
6,Dec 28 2022 1:56PM,253933,19,Paragon BioTeck
7,Dec 28 2022 1:32PM,253930,10,"Methapharm, Inc."
8,Dec 28 2022 1:28PM,253929,10,Emerginnova
9,Dec 28 2022 1:24PM,253927,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc.",,,1
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",,,1
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",,,1
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",,,1
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,,,1
5,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC",,,1
6,Dec 28 2022 1:56PM,253933,19,Paragon BioTeck,,,1
7,Dec 28 2022 1:32PM,253930,10,"Methapharm, Inc.",,,1
8,Dec 28 2022 1:28PM,253929,10,Emerginnova,,,1
9,Dec 28 2022 1:24PM,253927,10,Yusen – 3D Matrix,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc.",1,,
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",1,,
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",1,,
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,1,,
5,Dec 28 2022 1:57PM,253934,19,"NAPP Technologies, LLC",1,,
6,Dec 28 2022 1:56PM,253933,19,Paragon BioTeck,1,,
7,Dec 28 2022 1:32PM,253930,10,"Methapharm, Inc.",1,,
8,Dec 28 2022 1:28PM,253929,10,Emerginnova,1,,
9,Dec 28 2022 1:24PM,253927,10,Yusen – 3D Matrix,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc.",1,,
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",1,,
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",1,,
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 2:40PM,253941,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Dec 28 2022 2:36PM,253938,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0
2,Dec 28 2022 2:27PM,253937,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0
3,Dec 28 2022 2:23PM,253936,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Dec 28 2022 2:17PM,253935,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8632622,269.0,3.0,0.0
19,2285165,34.0,0.0,8.0
21,253916,1.0,0.0,0.0
22,253941,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8632622,269.0,3.0,0.0
1,19,2285165,34.0,0.0,8.0
2,21,253916,1.0,0.0,0.0
3,22,253941,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,269.0,3.0,0.0
1,19,34.0,0.0,8.0
2,21,1.0,0.0,0.0
3,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,269.0
1,19,Released,34.0
2,21,Released,1.0
3,22,Released,1.0
4,10,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,21,22
Status,,,,
Completed,0.0,8.0,0.0,0.0
Executing,3.0,0.0,0.0,0.0
Released,269.0,34.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,21,22
0,Completed,0.0,8.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0
2,Released,269.0,34.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,21,22
0,Completed,0.0,8.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0
2,Released,269.0,34.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()